In [13]:
import os
import ibis
from ibis import _
import ibis.selectors as s
from cng.utils import *
from cng.h3 import *
from minio import Minio
import streamlit 
from datetime import timedelta
import re
duckdb_install_h3()

# con = ibis.duckdb.connect("duck.db",extensions = ["spatial", "h3"])
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
set_secrets(con)

# Get signed URLs to access license-controlled layers
key = st.secrets["MINIO_KEY"]
secret = st.secrets["MINIO_SECRET"]
client = Minio("minio.carlboettiger.info", key, secret)

mobi_z8 = con.read_parquet("https://minio.carlboettiger.info/public-mobi/hex/all-richness-h8.parquet").select("h8", "Z").rename(richness = "Z")
svi_z8 = con.read_parquet("https://minio.carlboettiger.info/public-social-vulnerability/2022/SVI2022_US_tract_h3_z8.parquet").select("h8", "svi").filter(_.svi > 0)
carbon_z8 = con.read_parquet("https://minio.carlboettiger.info/public-carbon/hex/us-tracts-vuln-total-carbon-2018-h8.parquet").select('carbon','h8')
county_bounds = con.read_parquet("https://minio.carlboettiger.info/public-census/2024/county/2024_us_county.parquet")
tpl_z8 = con.read_parquet("s3://shared-tpl/conservation_almanac/z8/tpl_h3_z8.parquet")
landvote_z8 = con.read_parquet("s3://shared-tpl/landvote/z8/landvote_h3_z8.parquet")

In [15]:
current_tables = con.list_tables()

if "conservation_almanac" not in set(current_tables):
    con.create_table("conservation_almanac", tpl_z8)

if "landvote" not in set(current_tables):
    con.create_table("landvote", landvote_z8)

if "carbon" not in set(current_tables):
    con.create_table("carbon", carbon_z8)

if "mobi" not in set(current_tables):
    con.create_table("mobi", mobi_z8)

if "svi" not in set(current_tables):
    con.create_table("svi", svi_z8)

conservation_almanac = con.table("conservation_almanac")
landvote = con.table("landvote")
carbon = con.table("carbon")
mobi = con.table("mobi")
svi = con.table("svi")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
con.sql('''
SELECT fid, AVG(carbon) as mean_carbon
    FROM conservation_almanac
    LEFT JOIN carbon
    USING (h8)
    GROUP BY fid
    ORDER BY mean_carbon DESC LIMIT 10;
    ''').execute()

,fid,mean_carbon
0,12347,484.333333
1,21514,440.160714
2,14997,440.160714
3,29331,430.705128
4,29333,388.551515
5,29245,366.432727
6,25881,339.250000
7,24728,339.000000
8,24727,339.000000
9,29326,333.180000
